# Read me

Hello, This notebook for users who want to know how you can use the csv dataset and image dataset at the same time.

If you has a problem that it is hard to use two dataset, you will get some ideas from my notebook.

The data handling procedure will be skipped. If you want to know how it worked in detail, just clik on my previous notebook (https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners).

If you have any questions, please leave the comments.

I hope you to gain more imformation about data handling, DNN, CNN, and etc..

## **Knowledge can be improved by being shared.**

Please upvote!!


## [You can learn more skills for handling dataset or neural network.]

### [Parallel combination DNN with CNN] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/parallel-dnn-and-cnn-network-for-beginners
 
### [Image data handling without memory exploded] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners

### [Data handling & Deep learning] - Titanic competition (best score!!)
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
### [Deep learning model with SeLU activation function] - Titanic competition
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

### [Preparing a completed dataset with proper imputation method] - Titanic competition
 - https://www.kaggle.com/pythonash/making-completed-dataset

**Let's start!**

# Just run this code before you set your model.

- This code is for preparing dataset to input at your model.

- The details are described as in my notebook, "Image data handling without memory exploded".


## The difference from previous notebook is

> we will use sigmoid function as setting the output like binary classification.

> That is, we will normalize the target variable by dividing with 100 (the maximum value).

> So, the result will be distributed like probability.

> Finally, you have to transform the predict value by multiplying with 100.

In [ ]:
##################################JUST RUN THIS CODE FOR PREPARING DATASET##################################
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

os.chdir('../input/petfinder-pawpularity-score/train')

size_data = pd.DataFrame()
for file in os.listdir():
    imgg = cv2.imread(file)
    w,h,c = imgg.shape
    size_data=size_data.append([[w,h,c,imgg.size/3]])
    

train_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    train_img.append(file/255)


train_img_name = []
for i in os.listdir():
    train_img_name.append(i)


train_csv_data = pd.DataFrame()
for img, name in zip(train_img, train_img_name):
    name=name[:-4]
    location = train_csv[train_csv['Id'] == name].index[0]
    train_csv_data= train_csv_data.append([train_csv.loc[location]])

os.chdir('../test')

test_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    test_img.append(file/255)

test_img_name = []
for i in os.listdir():
    test_img_name.append(i)

test_csv_data = pd.DataFrame()
for img, name in zip(test_img, test_img_name):
    name=name[:-4]
    location = test_csv[test_csv['Id'] == name].index[0]
    test_csv_data= test_csv_data.append([test_csv.loc[location]])
test_csv_data=test_csv_data.reset_index().drop(['index'],axis=1)

train_csv_x = train_csv_data.drop(['Id','Pawpularity'],axis=1)
train_y = train_csv_data['Pawpularity']/100
test_csv_x = test_csv_data.drop(['Id'],axis=1)

In [ ]:
plt.imshow(test_img[0])

# The comnibation of DNN and CNN

We will use a combination structure which consists of DNN for CSV and CNN for IMAGE.

The idea is very simple.

1. Set the DNN model and CNN model, respectively.

2. Combinate two models with one outputs.

> This model will output one results through using CSV result and IMG result.

How can it work??

Let's start!

## Set your model

In this part, please read the codes carefully, we will use two input layers.

In [ ]:
##################### CSV FILE INPUT & IMG FILE INPUT LAYER ###########################
csv_input = tf.keras.Input(shape = train_csv_x.shape[1:], name = 'CSV_Input')        ##
img_input = tf.keras.Input(shape = np.array(train_img).shape[1:], name = 'IMG_Input')##
#######################################################################################
                                        ##
                                        ##
                                        ##
##################### CSV FILE HIDDEN LAYER STRUCTURE  ######################################
csv_hidden1 = tf.keras.layers.Dense(8, activation='relu', name='CSV_Hidden1')(csv_input)   ##
csv_hidden2 = tf.keras.layers.Dense(30, activation='relu', name='CSV_Hidden2')(csv_hidden1)##
csv_hidden3 = tf.keras.layers.Dense(50, activation='relu', name='CSV_Hidden3')(csv_hidden2)##
csv_dropout = tf.keras.layers.Dropout(0.5, name ='CSV_Dropout')(csv_hidden3)               ##
csv_hidden4 = tf.keras.layers.Dense(30, activation='relu', name='CSV_Hidden4')(csv_dropout)##
csv_hidden5 = tf.keras.layers.Dense(10, activation='relu', name='CSV_Hidden5')(csv_hidden4)##
#############################################################################################
                                         #
                                         #
                                         #
##################### IMG FILE CONVOLUTIONAL LAYER STRUCTURE  ######################################
img_conv1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 5, strides= 1, padding = 'same',   ##
                                   activation = 'relu', name='IMG_Conv1')(img_input)              ##
img_pool1 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool1')(img_conv1)                           ##
img_conv2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides= 1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv2')(img_pool1)              ##
img_conv3 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv3')(img_conv2)              ##
img_pool2 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool2')(img_conv3)                           ##
img_conv4 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv4')(img_pool2)              ##
img_pool3 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool3')(img_conv4)                           ##
img_conv5 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv5')(img_pool3)              ##
img_conv6 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides =1, padding = 'same',  ##
                                   activation = 'relu', name='IMG_Conv6')(img_conv5)              ##
img_pool4 = tf.keras.layers.MaxPool2D(2, name = 'IMG_Pool4')(img_conv6)                           ##
img_flatten = tf.keras.layers.Flatten(name = 'IMG_Flatten')(img_pool4)                            ##
img_dense1 = tf.keras.layers.Dense(3000, activation = 'relu', name='IMG_Dense1')(img_flatten)     ##
img_dropout1 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout1')(img_dense1)                      ##
img_dense2 = tf.keras.layers.Dense(3000, activation = 'relu', name='IMG_Dense2')(img_dropout1)    ##
img_dropout2 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout2')(img_dense2)                      ##
####################################################################################################
                                        ##
                                        ##
                                        ##
######################################### IMG & CSV DATA CONNETION #####################################
img_csv_concat = tf.keras.layers.Concatenate(name = 'connection', axis=1)([csv_hidden5, img_dropout2])##
########################################################################################################
                                        ##
                                        ##
                                        ##
################################# IMG & CSV DENSE LAYER AND DROPOUT #######################################
img_csv_dense1 = tf.keras.layers.Dense(100, name = 'IMG_CSV_Dense1', activation = 'relu')(img_csv_concat)##
img_csv_dropout = tf.keras.layers.Dropout(0.5, name = 'IMG_CSV_Dropout')(img_csv_dense1)                 ##
img_csv_dense2 = tf.keras.layers.Dense(50, name = 'IMG_CSV_Dense2', activation = 'relu')(img_csv_dropout)##
###########################################################################################################
                                        ##
                                        ##
                                        ##
################################# IMG & CSV DENSE LAYER AND DROPOUT ########################################
img_csv_output = tf.keras.layers.Dense(1, name = 'IMG_CSV_Output', activation = 'sigmoid')(img_csv_dense2)##
############################################################################################################
                                        ##
                                        ##
                                        ##
############################################# MODEL SETTING  ####################################################
model = tf.keras.Model(inputs=[csv_input, img_input], outputs=[img_csv_output], name='Pythonash_model')##
#################################################################################################################

## model summary

It shows that your model structure, simply.

In [ ]:
model.summary()

## plot your model

This plot shows your model and you can figure out you model structure, intuitively.

You can see that DNN model and CNN model are parallel and only one output layer is in this structure.

In [ ]:
# move you current directory to back.
os.chdir('../')
os.chdir('../')
os.chdir('../')
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

## Compile and fit your model.

In [ ]:
# We will use the learning schedule that decrease step by step.

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=500,
    decay_rate=0.3
)
opt = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics = tf.keras.metrics.RootMeanSquaredError())

epoch_number = 100


# It is used for preventing this model from overfitting.
# It remembers the best model in terms of validation loss.
# So, you have to load your best model when you attempt to predict.
check_1 = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only=True, verbose=2)

In [ ]:
model.fit( 
    x= [train_csv_x, np.array(train_img)], y = train_y, epochs=epoch_number,
    validation_split=0.2, verbose =2, workers=3, batch_size = 20, validation_batch_size = 20,
    callbacks = [check_1])

## Loading best model & submission

In [ ]:
best_model = tf.keras.models.load_model('pythonash_model.h5')
img_result = best_model.predict([test_csv_x,np.array(test_img)])
final_result = pd.DataFrame(img_result)
final_result.columns =['Pawpularity']
final_result

## Submit your result.

It has done!

I think it might be easy to you who already know about DNN and CNN.

In [ ]:
for ids, paw in zip(test_csv_data['Id'], final_result['Pawpularity']):
    location = submission[submission['Id'] == ids].index[0]
    submission['Pawpularity'].loc[location] = paw * 100
submission.to_csv('./working/submission.csv',index=False)
submission


# It's your turn!!

You have many opportunities that you can change this model parameters and get your submission score.

I recommend that you change the hyper parameters such as learning_rate, batch_size, activation function, the number of neurons, layers, and so on...

If you get any helps from my notebook, please upvote!!

Fingers crossed!!